In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [38]:
embeddings.embed_query("hello AI")

[-0.033388201147317886,
 0.034539803862571716,
 0.059474579989910126,
 0.05928615480661392,
 -0.06353538483381271,
 -0.06819586455821991,
 0.08823323249816895,
 0.0344407856464386,
 -0.03278515860438347,
 -0.015814995393157005,
 0.020981667563319206,
 -0.018340280279517174,
 -0.03983214125037193,
 -0.08047076314687729,
 -0.014469259418547153,
 0.03326486423611641,
 0.01425926387310028,
 -0.03404999524354935,
 -0.142915740609169,
 -0.023083265870809555,
 -0.02138010412454605,
 0.0026335634756833315,
 -0.04729275032877922,
 -0.010752726346254349,
 -0.06866799294948578,
 0.03112499974668026,
 0.07594592869281769,
 0.0011283049825578928,
 0.011631982401013374,
 -0.036039214581251144,
 0.04483765363693237,
 0.01839079149067402,
 0.12672799825668335,
 -0.001359735382720828,
 0.008206715807318687,
 0.06909963488578796,
 -0.0807635709643364,
 -0.05841311439871788,
 0.05375451594591141,
 0.026227623224258423,
 -0.00682859867811203,
 -0.05635840445756912,
 0.0032929950393736362,
 -0.072501808404

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [40]:
my_query="Narendra modi is prime minister of india?"

In [41]:
document_embedding=embeddings.embed_documents(documents)

In [42]:
document_embedding

[[0.11998698115348816,
  -0.021302608773112297,
  -0.04288088157773018,
  0.06645584106445312,
  -0.0643523558974266,
  -0.04424867779016495,
  0.022408470511436462,
  -0.049873046576976776,
  -0.023437676951289177,
  -0.03397207707166672,
  -0.014048065058887005,
  -0.06065930798649788,
  -0.003906784579157829,
  -0.017782047390937805,
  -0.047971028834581375,
  -0.06668160855770111,
  0.004103216342628002,
  -0.013092760927975178,
  0.04439772665500641,
  0.022350657731294632,
  0.009459580294787884,
  -0.02056453377008438,
  -0.0003356038359925151,
  -0.005685813259333372,
  0.05558698996901512,
  0.02512320503592491,
  -0.002817113883793354,
  0.00875900313258171,
  0.003255280666053295,
  -0.015963423997163773,
  0.014263689517974854,
  -0.11220851540565491,
  0.08968556672334671,
  -0.031083721667528152,
  -0.02422383800148964,
  0.006152089219540358,
  0.08058717846870422,
  0.01824996992945671,
  0.05568312108516693,
  0.01670268550515175,
  0.01589597947895527,
  0.00034103207

In [43]:
query_embedding=embeddings.embed_query(my_query)

In [44]:
len(query_embedding)

384

In [45]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756665, 0.3432456 , 0.8141324 ]])

In [46]:
from sklearn.metrics.pairwise import euclidean_distances

In [47]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848288, 1.14608416, 0.60970094]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [48]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [49]:
index=faiss.IndexFlatL2(384)
#384 size of embedding check lin eno 44
#len(query_embedding)

In [50]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001D8C61D08A0> >

In [51]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [52]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['26e92d68-df77-4e7b-a809-a70effcd9827',
 'cf975abd-4bc4-4c68-87f6-dd7cba9180fc',
 'a84a5e85-fcbb-4269-8556-a4eb78507147']

In [53]:
vector_store.index_to_docstore_id

{0: '26e92d68-df77-4e7b-a809-a70effcd9827',
 1: 'cf975abd-4bc4-4c68-87f6-dd7cba9180fc',
 2: 'a84a5e85-fcbb-4269-8556-a4eb78507147'}

In [54]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [55]:
results


[Document(id='cf975abd-4bc4-4c68-87f6-dd7cba9180fc', metadata={}, page_content='AI is powerful'),
 Document(id='26e92d68-df77-4e7b-a809-a70effcd9827', metadata={}, page_content='AI is future'),
 Document(id='a84a5e85-fcbb-4269-8556-a4eb78507147', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [64]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [65]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [66]:
vector_store.add_documents(documents=documents)

['bc422bb4-f405-4d2f-932a-c18c1fc5fc51',
 '5eca4bfe-7411-414d-9830-73a2c6b83f86',
 'da2d4036-1adc-4c08-845c-50649a5893bd',
 '3834e142-c272-41a0-9f01-5b94359df9d0',
 '9f4ce3d3-4932-4dc3-bda0-599f5cd74bae',
 'a9aae728-e2ff-458d-926c-77a3b1a0ee68',
 '29cab44e-a700-4af2-9906-78da7ed5832e',
 '4c9efb42-4ed6-4bd0-bb62-91faa5b17728',
 '08753aad-cd05-483e-a7a5-04bd5a7db666',
 'b898bcb2-30a3-4e87-a041-41929d02e757']

In [67]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='da2d4036-1adc-4c08-845c-50649a5893bd', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4c9efb42-4ed6-4bd0-bb62-91faa5b17728', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [68]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='da2d4036-1adc-4c08-845c-50649a5893bd', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4c9efb42-4ed6-4bd0-bb62-91faa5b17728', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='b898bcb2-30a3-4e87-a041-41929d02e757', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='bc422bb4-f405-4d2f-932a-c18c1fc5fc51', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [73]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)
result

[Document(id='3834e142-c272-41a0-9f01-5b94359df9d0', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='5eca4bfe-7411-414d-9830-73a2c6b83f86', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(id='08753aad-cd05-483e-a7a5-04bd5a7db666', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

In [70]:
result[0].metadata

{'source': 'news'}

In [71]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [74]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [75]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='da2d4036-1adc-4c08-845c-50649a5893bd', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4c9efb42-4ed6-4bd0-bb62-91faa5b17728', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='b898bcb2-30a3-4e87-a041-41929d02e757', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [37]:
# Example placeholders for storage types (not executable code)
# inmemory(server)
# ondisk(server)
# cloud(yet to discuss)

In [77]:
vector_store.save_local("today's class faiss index")

In [78]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [79]:
new_vector_store.similarity_search("langchain")

[Document(id='da2d4036-1adc-4c08-845c-50649a5893bd', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4c9efb42-4ed6-4bd0-bb62-91faa5b17728', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='9f4ce3d3-4932-4dc3-bda0-599f5cd74bae', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='29cab44e-a700-4af2-9906-78da7ed5832e', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [80]:
from langchain_community.document_loaders import PyPDFLoader

In [81]:
FILE_PATH=r"C:\AIandGenAI_Krish_May25\llama2-document.pdf"

In [82]:
loader=PyPDFLoader(FILE_PATH)

In [83]:
len(loader.load())

77

In [84]:
pages=loader.load()

In [85]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [86]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [87]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [88]:
split_docs = splitter.split_documents(pages)

In [89]:
len(split_docs)

615

In [90]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [91]:
vector_store.add_documents(documents=split_docs)

['edf2120a-ff7f-4881-9172-8a897eca12e5',
 '4c0740c2-cf6b-439b-a894-91d5c5cd4aa2',
 '2fd13881-bd2d-42e2-9381-71376402256e',
 'ba834ff1-3d99-4422-af54-74cca18ff7f5',
 '5e475d1f-4c52-40f8-af5a-f2f333ad2e5b',
 'af4920b2-8899-4df6-90a6-7b0c0189d6c6',
 '90984d1a-3fb7-4e24-aaea-4c678150a09f',
 'fac07f87-9038-4071-8d6c-461d999b2569',
 '16b686d5-4c6f-42b7-9f74-73cba29080f9',
 '3fe587d4-bd9f-444e-b41f-8548f14e84e6',
 '2bd0ca35-78f2-43fe-adb8-2972e8129c76',
 'fa78de19-a39c-453c-be3d-467c900ed594',
 '8da2307a-5cc0-4983-843d-00f6b27d94c3',
 'fab15dcb-9152-484a-9dfd-ad8bc5b8c0f0',
 '36aacae3-6051-40ed-8f5c-2970302586b4',
 '1ef5e43f-9ae4-4c8d-b314-3a619d044163',
 '03530367-aca0-4001-8968-2072356e800e',
 '0f03a00a-d979-485f-8e56-5dfe0e93ad91',
 '3cf94fc2-5fe6-47fa-a3e9-383150590e0b',
 'ef607257-8667-475b-bb9a-379cd651232a',
 'd40c82e5-3919-4c2a-b4ca-17571d98d66d',
 '53d290e6-932f-4556-a279-9c35f117a906',
 '4a582d5b-ffb3-43c6-8b6e-0099b0241ffa',
 '4dc9b51d-97a1-48a8-ba83-7bea6d174bbf',
 '92234b97-3141-

In [92]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [93]:
retriever.invoke("what is llama model?")

[Document(id='92234b97-3141-4e96-b451-8e5a1070a26f', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\AIandGenAI_Krish_May25\\llama2-document.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='27755ac3-c5af-4086-9aab-eac0123e6d65', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords

In [94]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [95]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [96]:
import pprint


In [97]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [98]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [100]:
# Example: Compose the RAG chain using the defined components
# (see next cell for actual usage)
{"context": retriever, "prompt": prompt, "model": model, "parser": StrOutputParser()}

{'context': VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D8F8207260>, search_kwargs={'k': 10}),
 'prompt': ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]),
 'model': ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=S

In [101]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [102]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [103]:
rag_chain.invoke("what is llama model?")

'Llama is a large language model (LLM) developed by Meta.  It has various versions, including tuned models for chat and pretrained models adaptable to different natural language tasks.  Llama 2 is a newer iteration with improved performance and safety features compared to Llama 1.'